# Specification

- This is a command Line program using standard parameters, STDIN and STDOUT redirection
- Program name: missingMotif.py
- Input/Output: missingMotif.py < [input file] > [output file]

## options:

- --minMotif  minimum motif size to evaluate (int)
- --maxMotif maximum motif size to evaluate (int)
- --cutoff  Z-score cutoff (float)

Ex: missingMotif.py --minMotif=3 --maxMotif=8 --cutoff=-4.0 < [input file] > [output file]

# Background
Many prokaryotic genomes encode Restriction-Modification systems (RM) that act to protect the cell from invading DNA by cutting at specific sites (frequently short 4-6 base reverse complement palindromes). They also protect host DNA from being cut by modifying any sites within the host DNA that would be sites for the cutting enzyme. It is not unusual to find that these enzymes are adjacent to each other in the host genome. (see: https://en.wikipedia.org/wiki/Restriction_modification_system).

Meselson and Stahl (1958) showed that replication of the bacterial dsDNA genome happens in a semi-conservative fashion (see: https://en.wikipedia.org/wiki/Prokaryotic_DNA_replication). The resulting circular chromosome(s) would then be transiently hemi-methylated because the new daughter strand may not been have exposed to the protective methylation of the RM system before its cutting activity arrives. This creates a race between RM cutting and protecting activities may expose the cell to strand breaks.

Is a selective advantage incurred for the cell by allowing mutations to accumulate at these sites? If so, then would we expect the host genome to become depleted of RM sites over time? Can we use this depletion process as a signature for the existence of the specific RM systems present in the genome? 

How can we find sequences that are missing in a genome?

Note: "Stealth" has recently been accepted for publication, and is now publicly available at: https://git.ucsc.edu/dbernick/stealth



# Assignment

Write a proper style, commandLine BME205 solution that reads a fasta file and ranks motifs based on how statistically underrepresented the specific motif is. We need only consider sequence motifs that are up to about 8 in length ( this is a program option). In order to use the equation that we developed in class, the minimum size should be at least 3 ( this is an option also). We will find that there are quite a few of these, so we need to specify a statistical cutoff. For this assignment, we will use Z-scores. The scores we are looking for will be negative since we are looking for underrepresented sequences (this cutoff is a program option too). Finally, we need to sort the output by decreasing motif size and then increasing (most negative to least negative) z-score within motif size groups and print it out.

Remember that sequences that we get from DNA sequencing are from only one strand of DNA. This means that when we see a AAGGTT, this implies that AACCTT (the reverse complement) is present on the other strand. We should count these sequences as equivalent. 

Your report should be sorted by Zscore within motif-Size, such that the longest motifs are at the beginning of the report and, within motifs of a given size, print the most significant motifs first. Print the k-mer pairs in alpha order. ( AAA:TTT and not TTT:AAA)

The extra credit for this assignment is particularly interesting. If you have time, this will present better results. 

Here is an example (non extra credit):

In [ ]:
sequence: reverse     count      Expect Zscore
AAAATTTA:TAAATTTT	835	1737.66	-21.66
GATTAATA:TATTAATC	550	1326.89	-21.33
AATTAATA:TATTAATT	929	1839.72	-21.24
AATTAATC:GATTAATT	977	1861.79	-20.51
ATAATTAA:TTAATTAT	1033	1926.49	-20.36
GAAATTTA:TAAATTTC	378	1031.07	-20.34
CCGATCGC:GCGATCGG	1323	2284.74	-20.12
GCGATCGA:TCGATCGC	1221	2121.78	-19.56
ACGATCGC:GCGATCGT	1188	2035.31	-18.78
GTTTAAAA:TTTTAAAC	519	1151.17	-18.63
AAATATTA:TAATATTT	738	1444.79	-18.60
AAAATTTG:CAAATTTT	857	1591.95	-18.42
CATTAATC:GATTAATG	479	1037.49	-17.34
ATATATAA:TTATATAT	271	736.19	-17.15
ATTTAAAG:CTTTAAAT	419	941.16	-17.02
AAATATTG:CAATATTT	687	1287.74	-16.74
CAAATTTA:TAAATTTG	670	1265.24	-16.74
CTTTAAAC:GTTTAAAG	468	995.13	-16.71

# Hints
- For **genomes that include multiple fasta records, do NOT concatenate them**. They should be reported as a single genome, though we do not want to create false kmers that would result from concatenation
- **Avoid zero divide errors!** Close examination of our null model will show that if we have a zero in the denominator, we will have a zero in the numerator. If you have pre-populated your count dictionary, then you will need to handle dividing by zero.
- Only consider kmers that are constructed from the **canonical bases {A, C, G, T}**.
- Z-score is calculated as: $\frac{s-\mu}{sd}$, where s is the specific count you are converting, and mean and standard deviation are given by µ and sd. Both µ and sd describe the null distribution. In our case, we can treat the expected distribution as a binomial where: $\mu=np,sd=\sqrt{np(1-p)}$. n = genome size (approximately), and p = probability of success, given the null distribution.
- c(x) defined as the count of x, generally the count of some substring. For example, for kmers of size 4: $K=k_{1}k_{2}k_{3}k_{4}, 
Pr(K)=\frac{c(K)}{N},
E(K) = \frac{c(k_{1}k_{2}k_{3})\cdot c(k_{2}k_{3}k_{4})}{c(k_{2}k_{3})}$
(see below for the derivation of our null model) We can then calculate n and p, and derive µ and sd. 
- To keep your scores lined up, use tabs to separate values in each line. The following format string will be of use: 

print('{0:8}:{1:8}\t{2:0d}\t{3:0.2f}\t{4:0.2f}'.format(seq, rSeq, count,E,pVal))


# Null Model derivation
If we assume complete independence of the any of the characters that make up a sequence K, we could approximate the expected count of K, E(K),  as:
$\begin{matrix} 
K=k_{1}k_{2}k_{3}k_{4}, Pr(k_{i})=\frac{c(k_{i})}{N} \\ 
E(K)=N * Pr(K) = N * 
Pr(k_{1})
Pr(k_{2})
Pr(k_{3})
Pr(k_{4}) 
\end{matrix} 
$

the above is a markov(0) approximation for the expected count (E) of our sequence K.

Rarely is this assumption of independence a good null model, due to many factors including codon bias which drives selection of 3-mers based on underlying amino acid frequencies. We can model this dependence using a markov model, where the probability associated with any symbol is conditioned on the preceding characters. For a kmer of size 4, we can condition our character probabilities using the preceding two bases without requiring use of the count of the specific 4-mer that we are trying to approximate(see note on information content). We need to consider edge conditions in our approximation. In general, we can use a model of order k-2, where k is the size of the kmer as long as we have sufficient data, therefore:

$
\begin{matrix} 
Pr(K)=\frac{E(K)}{N} = 
Pr(k_{1})
Pr(k_{2}|k_{1})
Pr(k_{3}|k_{1}k_{2})
Pr(k_{4}|k_{2}k_{3}) \\
= \frac{c(k_{1})}{N}
\frac{c(k_{1}k_{2})}{c(k_{1})}
\frac{c(k_{1} k_{2} k_{3})} {c(k_{1} k_{2})}
\frac{c(k_{2} k_{3} k_{4})} {c(k_{2} k_{3})} \\
= \frac{1}{N}
\frac{c(k_{1} k_{2} k_{3})c(k_{2} k_{3} k_{4})} {c(k_{2} k_{3})}
\end{matrix}
$

note: the conditional probability Pr(a | b) is read as the probability of finding sequence a, given sequence b. In this case we treat sequence b as adjacent and left of a, and ask what is the probability of finding sequence a, given that b is immediately to the left of a. This means that we should count the occurrences of string ba, among all strings that start with b (taken as the count of all sequence b).

note on information content: What would happen if we were trying to predict the frequencies of 4-mers, and our null model contained the frequency of 4-mers? 

2021 Note: Make sure to review the module video on this. The derivation is a bit more elaborate.

# Notes
- I removed all extra credit from this assignment. It's full enough as is.
- I considered adding pseudo counts to this assignment but have deferred that to the next (4-Oct)
- Added the full derivation of the null model (4-Oct)
- clarified the use of E - the expected value, and in our case, an expected motif count. (4-Oct)
- clarified the use of the count function c(x) (4-Oct)
- clarified the use of counts of strings in conditional probabilities. (4-Oct)
- clarified the avoidance of zero divide or the use of kmers that contain non-canonical bases (4-Oct)

## 2018 additions

- binned the reverse complement sequences. This produces better Evalues and a more concise report. ( 1-Oct)

## 2020 addition

- its good to think about our kmer counting by asking how many names does this kmer have? For example, the kmer AAAA when viewed from the opposite strand is TTTT. This bit of data has 2 names, so when we are counting we make sure that counts of this kmer by either of its names goes into the same bin. How should we handle a reverse complement palindrome like TATA ? When viewed from either strand, it has the same name.

## 2021 addition:

- we are using notebooks now to formalize inspections and promote Python learning.

## 2023
- required submissions to be .py files with command line options, STDIN and STDOUT as input/output
- provided example of published solution at: https://git.ucsc.edu/dbernick/stealth


## Inspection Intro
Provide design level information for your inspection team here.Examples:
- How did you count kmers of each size without having to read the data multiple times?
I iterated over the whole sequence with a nested for loop while adding it to the genome object. A common thing in other people's code that I saw was that they included catch statements to prevent key errors in dictionaries. I resolved this using the power of defaultdict.

- How did you organize the computing of mean and SD for each of the kmer groups ?


- How did you compute Z-scores for each kMer in a kMer group?
Z-scores were calculated by using the expected count of the kmer then using that value in the formula. Following stealth's implementation of the function, there is a safety net which provides a default value for calculating the standard deviation so that there is no division by 0.

- How did you sort the final report to include most significant Z-scores at the top of each kmer-size group?
Kmers and relevant data from the genome object were appended to a list, where a .sort() function organizes the list by kmer length in descending then by Z-score in ascending order.

- How did you organize the program ?
Functions were stored in the class.


## Inspection Results
- What did your inspection team find? Which of the inspection results did you use, how did you resolve them, and which did you decide against? Who was on your inspection team?

Before my code was complete me and my team found that I often encountered dictionary size-change errors while iterating over my code's kmer-count dict. It's a hacky solution but I simply copied over the dictionary before I iterated over it so that defaultdict wouldn't cause any errors.

# Code that may be useful

In [1]:
import sys
import itertools, math
from collections import defaultdict

class FastAreader:
    def __init__(self, fname=''):
        '''contructor: saves attribute fname '''

        self.fname = fname
        self.fileH = None

    def doOpen(self):
        if self.fname == '':
            return sys.stdin
        else:
            return open(self.fname)

    def readFasta(self):

        header = ''
        sequence = ''

        with self.doOpen() as self.fileH:

            header = ''
            sequence = ''

            # skip to first fasta header
            line = self.fileH.readline()
            while not line.startswith('>'):
                line = self.fileH.readline()
            header = line[1:].rstrip()

            for line in self.fileH:
                if line.startswith('>'):
                    yield header, sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else:
                    sequence += ''.join(line.rstrip().split()).upper()

        yield header, sequence


In [2]:
class CommandLine():
    '''
    Handle the command line, usage and help requests.

    CommandLine uses argparse, now standard in 2.7 and beyond. 
    it implements a standard command line argument parser with various argument options,
    a standard usage and help, and an error termination mechanism do-usage_and_die.

    attributes:
    all arguments received from the commandline using .add_argument will be
    avalable within the .args attribute of object instantiated from CommandLine.
    For example, if myCommandLine is an object of the class, and requiredbool was
    set as an option using add_argument, then myCommandLine.args.requiredbool will
    name that option.

    '''

    def __init__(self, inOpts=None):
        '''
        CommandLine constructor.
        Implements a parser to interpret the command line argv string using argparse.
        '''

        import argparse
        self.parser = argparse.ArgumentParser(
            description='Program prolog - a brief description of what this thing does',
            epilog='Program epilog - some other stuff you feel compelled to say',
            add_help=True,  # default is True
            prefix_chars='-',
            usage='%(prog)s [options] -option1[default] <input >output'
        )

        self.parser.add_argument('-l', '--minMotif', nargs='?', default=1, action='store',
                                 help='min kMer size ')
        self.parser.add_argument('-m', '--maxMotif', nargs='?', default=8, action='store',
                                 help='max kMer size ')
        self.parser.add_argument('-c', '--cutoff', nargs='?', type=float, default=.01, action='store',
                                 help='Zscore cutoff')


        self.parser.add_argument('-v', '--version', action='version', version='%(prog)s 0.1')
        if inOpts is None:
            self.args = self.parser.parse_args()
        else:
            self.args = self.parser.parse_args(inOpts)


In [65]:
class Genome:
    """ """
    
    valid_bases = 'ACGT'
    
    def __init__(self, minK, maxK, cutoff):  
        """Constructs genome object and creates a dictionary containing all possible kmers"""
        
        self.min = int(minK)
        self.max = int(maxK)
        self.cutoff = cutoff
        
        self.counts = defaultdict(lambda: 0)
        
        
        for k in range(self.min, self.max+1):
            newCount = {''.join(kmer):0 for kmer in itertools.product(Genome.valid_bases, repeat=k)}
            self.counts.update(newCount)
        
        self.n = sum(self.counts.values())
    
    def reverse_c(self, seq):
        '''Provides reverse complement of given sequence'''
        return seq[::-1].translate(str.maketrans('ACGTN', 'TGCAN'))

    def addSeq(self, seq):
        '''Iterates over sequence and counts kmers'''
        for k in range(self.min - 2, self.max + 1):
            for b in range(0, len(seq) - self.max):
                kmer = seq[b:b+k]
                rc_kmer = self.reverse_c(kmer)
                
                if kmer in self.counts:
                    self.counts[kmer] += 1 #I LOVE DEFAULT DICT!
                if (rc_kmer != kmer) and rc_kmer in self.counts:
                    self.counts[rc_kmer] += 1
                
        self.n += len(seq) - self.max #adds to the count
        
        
    def E(self,s):
        '''returns the expected count of a kmer, implementation taken from stealth ''' 
        try:
            return self.counts[s[:-1]] * self.counts[s[1:]] / self.counts[s[1:-1]]
        except KeyError:
            return self.counts[s]
        except ZeroDivisionError:
            return 0.

    def Zscore(self, s):
        '''returns the Z score of the kmer, implementation taken from stealth'''
        mu = self.E(s)
        var = mu*(1-mu/self.n)
        if var == 0.0: #Catches any zeroes before division
            return 1000000000
        return (self.counts[s] - mu)/math.sqrt(var)

    def Evalue(self,s):
        """Return expected kmer count for a sequence using a Markov (k-2) model. Implementation courtesy of Konstantinos."""
        try:
            # Multiply counts of each k-1mer and divide by the k-2mer that contains the middle sequnces.
            return self.counts[s[:-1]] * self.counts[s[1:]] / self.counts[s[1:-1]]
        except ZeroDivisionError:
            return 0.
        except KeyError:
            return self.counts[s]

        



        

In [67]:
def main (inFile=None, options = None):
    ''' Setup necessary objects, read data and print the final report.'''
    cl = CommandLine(options) # setup the command line
    sourceReader = FastAreader(inFile) # setup the Fasta reader Object
    thisGenome = Genome(cl.args.minMotif, cl.args.maxMotif, cl.args.cutoff) # setup a Genome object
    
    ###
    # Give your Genome some data and tell it to do stuff
    ###

    for head, seq in sourceReader.readFasta():
        thisGenome.addSeq(seq)

    copy_dict = {k: v for k, v in thisGenome.counts.items() if v}
    final_list = []
    for kmer, count in copy_dict.items():
        if thisGenome.Zscore(kmer) < thisGenome.cutoff:
            final_list.append([kmer, thisGenome.reverse_c(kmer), count, thisGenome.Evalue(kmer), thisGenome.Zscore(kmer)])

    final_list.sort(key=lambda e: (len(e[0]), -e[4]), reverse = True )
    print("N = ", thisGenome.n)
    for kmer in final_list:
        print('{0:8}:{1:8}\t{2:0d}\t{3:0.2f}\t{4:0.2f}'.format(kmer[0], kmer[1], kmer[2], kmer[3], kmer[4]))
    
if __name__ == "__main__":
    main(inFile="Ecoli-UMN026.fa",options=[ "--minMotif=3", "--maxMotif=8", "--cutoff=-4."])
   

N =  5202082
CGATATCG:CGATATCG	256	1127.14	-25.95
GGCGCGCC:GGCGCGCC	203	973.44	-24.70
GCGATCGC:GCGATCGC	249	1008.19	-23.91
GCGCGCGC:GCGCGCGC	208	839.20	-21.79
GCAGCTGC:GCAGCTGC	215	849.93	-21.78
CCAGCTGG:CCAGCTGG	163	760.33	-21.66
CGTTAACG:CGTTAACG	207	821.63	-21.44
CCGCGCGG:CCGCGCGG	192	779.87	-21.05
CTGCGCAG:CTGCGCAG	159	716.82	-20.84
AAATATTT:AAATATTT	180	744.09	-20.68
TATCGATA:TATCGATA	187	748.82	-20.53
GCAATTGC:GCAATTGC	189	751.43	-20.52
TTGCGCAA:TTGCGCAA	169	717.95	-20.49
ATCGCGAT:ATCGCGAT	210	772.15	-20.23
CACCGGTG:CACCGGTG	154	672.97	-20.01
TTTTAAAA:TTTTAAAA	152	668.40	-19.98
CGCGCGCG:CGCGCGCG	178	710.35	-19.98
CAGCGCTG:CAGCGCTG	128	623.27	-19.84
TACCGGTA:TACCGGTA	155	657.87	-19.61
GGTTAACC:GGTTAACC	157	659.30	-19.56
CAGGCCTG:CAGGCCTG	109	579.32	-19.54
CAATATTG:CAATATTG	181	688.24	-19.34
TGATATCA:TGATATCA	137	615.23	-19.28
AAAATTTT:AAAATTTT	161	649.76	-19.18
CATTAATG:CATTAATG	189	691.56	-19.11
ATCATGAT:ATCATGAT	155	635.81	-19.07
TTTATAAA:TTTATAAA	159	641.88	-19.06
AACGCGTT:AACG